In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
ham_cbg_allcombs_ts_5000 = pd.read_csv('./ham_cbg_allcombs_ts_5000.csv', index_col=0)
ham_cbg_allcombs_ts_750 = pd.read_csv('./ham_cbg_allcombs_ts_750.csv', index_col=0)

In [3]:
ham_cbg_allcombs_ts_5000 = ham_cbg_allcombs_ts_5000.dropna(subset=['transit_score'])
ham_cbg_allcombs_ts_750 = ham_cbg_allcombs_ts_750.dropna(subset=['transit_score'])

In [36]:
ham_lodes = pd.read_csv('./hamilton_lodes_2019.csv', dtype={"TRACTCE20_home":"string", "TRACTCE20_work":"string"})
ham_lodes.h_geocode = ham_lodes.h_geocode.apply(lambda x: int(x/1000))
ham_lodes.w_geocode = ham_lodes.w_geocode.apply(lambda x: int(x/1000))
ham_lodes.w_geocode = ham_lodes.w_geocode.astype(str)
ham_lodes.h_geocode = ham_lodes.h_geocode.astype(str)

ham_cbg = gpd.read_file('./ham_2020_cbg.geojson')
ham_cbg.geoid20 = ham_cbg.geoid20.astype(str)

In [37]:
ham_lodes.groupby('w_geocode').sum().reset_index().dropna(subset=['total_jobs']).merge(ham_cbg, left_on='w_geocode', right_on='geoid20')[['w_geocode', 'total_jobs', 'geometry']].to_csv('lodes_jobs_wgeo.csv', index=False)
ham_lodes.groupby('h_geocode').sum().reset_index().dropna(subset=['total_jobs']).merge(ham_cbg, left_on='h_geocode', right_on='geoid20')[['h_geocode', 'total_jobs', 'geometry']].to_csv('lodes_jobs_hgeo.csv', index=False)

In [7]:
pd.set_option('display.max_columns', None)
ham_lodes_merged = ham_lodes[['h_geocode', 'w_geocode', 'total_jobs']].merge(ham_cbg_allcombs_ts_5000, left_on=['h_geocode', 'w_geocode'], right_on=['home', 'work'], how='left')
ham_lodes_merged2 = ham_lodes[['h_geocode', 'w_geocode', 'total_jobs']].merge(ham_cbg_allcombs_ts_750, left_on=['h_geocode', 'w_geocode'], right_on=['home', 'work'], how='left')
# .drop(['h_geocode', 'w_geocode'], axis=1)
ham_lodes_cbg_ts = ham_lodes_merged.groupby('h_geocode').mean().reset_index()
ham_lodes_cbg_ts2 = ham_lodes_merged2.groupby('h_geocode').mean().reset_index()

In [8]:
ham_lodes_cbg_ts = ham_lodes_cbg_ts.drop(columns=['home', 'work', 'trip1', 'trip2', 'trip3'], axis=1)
ham_lodes_cbg_ts.h_geocode = ham_lodes_cbg_ts.h_geocode.astype(str)
ham_lodes_cbg_ts.w_geocode = ham_lodes_cbg_ts.w_geocode.astype(str)

ham_lodes_cbg_ts2 = ham_lodes_cbg_ts2.drop(columns=['home', 'work', 'trip1', 'trip2', 'trip3'], axis=1)
ham_lodes_cbg_ts2.h_geocode = ham_lodes_cbg_ts2.h_geocode.astype(str)
ham_lodes_cbg_ts2.w_geocode = ham_lodes_cbg_ts2.w_geocode.astype(str)
# ham_lodes_cbg_ts.dropna(subset=['transit_score'])

In [10]:
ham_lodes_cbg_ts = ham_lodes_cbg_ts.sort_values('h_geocode')
ham_lodes_cbg_ts2 = ham_lodes_cbg_ts2.sort_values('h_geocode')

In [74]:
ham_lodes_cbg_ts['total_jobs'] = ham_lodes.groupby('h_geocode').sum().reset_index().sort_values('h_geocode')['total_jobs']
ham_lodes_cbg_ts = ham_lodes_cbg_ts.drop('w_geocode', axis=1)
ham_lodes_cbg_ts2['total_jobs'] = ham_lodes.groupby('h_geocode').sum().reset_index().sort_values('h_geocode')['total_jobs']
ham_lodes_cbg_ts2 = ham_lodes_cbg_ts2.drop('w_geocode', axis=1)

In [76]:
ham_lodes_cbg_ts2

,h_geocode,total_jobs,transit_time,transfers,walk_percent,waiting_percent,bus_capacity_upto_2hours,drive_time_fn,walk_time_fn,transit_score
0,470650004001,366,2452.733333,1.116667,28.754831,8.167895,90.000000,876.700000,4640.916667,0.376727
1,470650004002,33,1596.785714,0.285714,38.684279,5.005637,90.000000,762.357143,3838.571429,0.520810
2,470650006001,166,2448.578947,0.947368,42.642025,5.301105,86.842105,785.736842,4129.157895,0.315649
3,470650006002,261,2270.125000,0.656250,40.832602,4.128254,90.000000,897.906250,4444.156250,0.385627
4,470650006003,182,2196.608696,0.750000,36.735096,11.660575,90.000000,817.119565,4289.521739,0.368921
...,...,...,...,...,...,...,...,...,...,...
216,470650123003,390,3588.892308,1.138462,12.821120,7.678264,90.000000,833.564103,4973.471795,0.247515
217,470650124001,117,1472.910714,0.321429,34.963064,6.958159,90.000000,879.089286,3960.928571,0.678804
218,470650124002,122,1267.500000,0.166667,59.895009,1.990415,90.000000,558.166667,2849.500000,0.428430
219,470650124003,194,1543.448276,0.551724,63.667074,5.189601,90.000000,489.724138,2425.000000,0.320819


In [77]:
ham_lodes_cbg_ts.to_csv('ham_lodes_cbg_ts_5000.csv', index=False)
ham_lodes_cbg_ts2.to_csv('ham_lodes_cbg_ts_750.csv', index=False)
# ham_lodes_cbg_ts.dropna(subset=['transit_score']).to_csv('ham_lodes_cbg_ts_filt.csv', index=False)